In [5]:
!pip uninstall tensorflow
!pip install tensorflow==1.15 # 1.15 버전 Tensorflow 설치
# !pip install tensorflow==1.14 # 1.14 버전 Tensorflow 설치

Found existing installation: tensorflow 1.15.0
Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
    /usr/local/lib/python3.7/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflo

In [7]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [8]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from google.colab import drive
# drive.mount('/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data')
drive.mount('/content/drive')

%cd drive/MyDrive/IITP/sohyun/creditcard_prediction/data

Mounted at /content/drive
/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data


In [10]:
import torch

import pandas as pd
import numpy as np

from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [11]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode :
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x), 0
        
    def __len__(self):
        return len(self.df)

# train_dataset = MyDataset(df=train_df, eval_mode=False)
# train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)

# val_dataset = MyDataset(df = val_df, eval_mode=True)
# val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False)

import pandas as pd
def get_loader(data_path, batch_size, mode='train'):
    """Build and return data loader."""
    train_df = pd.read_csv('./train.csv')
    train_df = train_df.drop(columns=['ID'])
    val_df = pd.read_csv('./val.csv')
    val_df = val_df.drop(columns=['ID'])
    test_df = pd.read_csv('./test.csv')
    test_df = test_df.drop(columns=['ID'])

    # dataset = MyDataset(data_path, mode)
    train_dataset = MyDataset(df=train_df, eval_mode=False)
    valid_dataset = MyDataset(df=val_df, eval_mode=True)
    test_dataset = MyDataset(df=test_df, eval_mode=False)
    shuffle = False
    if mode == 'train':
        shuffle = True

    data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=shuffle)
    data_loader_v = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=shuffle)
    data_loader_test = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=shuffle)
    return data_loader, data_loader_v, data_loader_test


In [12]:
train_df = pd.read_csv('./train.csv')
train_df = train_df.drop(columns=['ID'])
val_df = pd.read_csv('./val.csv')
val_df = val_df.drop(columns=['ID'])
test_df = pd.read_csv('./test.csv')
test_df = test_df.drop(columns=['ID'])

train_total_data_normal = train_df 
y_test_normal = val_df['Class']
test_total_data_normal = val_df.drop(columns=['Class'])

In [14]:
# cc_data = df.values    #convert pandas dataframe to numpy array
# cc_data.shape  # shape -> (284807, 30)
# print("cc_data.shape=", cc_data.shape)

#cc_data_normal = cc_data[cc_data[:,30]==0]   #with 'Time'
#cc_data_fraud = cc_data[cc_data[:,30]==1]

# cc_data_normal = cc_data[cc_data[:,29]==0]    #without 'Time'
# cc_data_fraud = cc_data[cc_data[:,29]==1]

# print("shape cc_data_normal", cc_data_normal.shape)    #shape -> (284315, 31)  normal examples
# print("shape cc_data_fraud", cc_data_fraud.shape)       #shape -> (492, 31)    fraud examples     

# from sklearn.model_selection import train_test_split

# # without Time feature
# train_total_data_normal, test_total_data_normal, y_train_normal, y_test_normal = train_test_split(
#     cc_data_normal, cc_data_normal[:,29], test_size=0.2, random_state=37)

#############################################################################################
import sklearn
import sklearn.pipeline
import sklearn.preprocessing

###############################################################
#expects input state shape = (batch_dim,2)  
#returns output 'scaled' shape = (batch_dim,2)
def process_state(state):
    scaled = scaler.transform(state)
    return scaled  
#################################################################

print("shape train_total_data_normal", train_total_data_normal.shape)       
total_training_instances = len(train_total_data_normal)
print(total_training_instances)

# test_total_data = np.concatenate((cc_data_fraud, test_total_data_normal), axis=0) #########################################################################

# print("shape test_total_data", test_total_data.shape) 


#n_epochs = 50

#z_dim = 1  #1   #z compression dimensionality
#K_dims = 3  # orig 3
#lamda1 = 0.1
#lamda2 = 0.01   # 0.01
#lr = 0.001   #0.001
#batch_size = 1024  # 256, 512, 1024

def dagmm(n_epochs=100, hp_drop=0.4, z_dim=1,K_dims=3,lamda1=0.1,lamda2=0.01,lr=0.001,
          batch_size=1024, n_hidden1=20, n_hidden2=10, n_hidden3=5, n_layer1=10):
    #######################################################################################
    
    import tensorflow as tf
    import numpy as np
    from scipy.optimize import minimize_scalar
    tf.reset_default_graph()
    training = tf.placeholder_with_default(False, shape=())   

    def encoder(x, z_dim, n_hidden1, n_hidden2, n_hidden3, dropout_prob):

        with tf.variable_scope("encoder"):
            init_xavier = tf.contrib.layers.xavier_initializer()
            he_init = tf.contrib.layers.variance_scaling_initializer()
            x_drop = tf.layers.dropout(x, dropout_prob, training=training)
            hidden1 = tf.layers.dense(x_drop, n_hidden1, tf.nn.tanh, he_init)
            hidden1_drop = tf.layers.dropout(hidden1, dropout_prob, training=training)
            hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, tf.nn.tanh, he_init)
            hidden2_drop = tf.layers.dropout(hidden2, dropout_prob, training=training)
            hidden3 = tf.layers.dense(hidden2_drop, n_hidden3, tf.nn.tanh, he_init)
            hidden3_drop = tf.layers.dropout(hidden3, dropout_prob, training=training)
            z_comp = tf.layers.dense(hidden3_drop,z_dim,None, he_init)
        return z_comp

    def decoder(z_comp, x_dim, n_hidden1, n_hidden2, n_hidden3, dropout_prob, reuse=False):

        with tf.variable_scope("decoder"):
            init_xavier = tf.contrib.layers.xavier_initializer()
            he_init = tf.contrib.layers.variance_scaling_initializer()
            hidden3 = tf.layers.dense(z_comp, n_hidden3, tf.nn.tanh, he_init)
            hidden3_drop = tf.layers.dropout(hidden3, dropout_prob, training=training)
            hidden2 = tf.layers.dense(hidden3_drop, n_hidden2, tf.nn.tanh, he_init)
            hidden2_drop = tf.layers.dropout(hidden2, dropout_prob, training=training)
            hidden1 = tf.layers.dense(hidden2_drop, n_hidden1, tf.nn.tanh, he_init)
            hidden1_drop = tf.layers.dropout(hidden1, dropout_prob, training=training)
            x_hat_logits = tf.layers.dense(hidden1_drop, x_dim, None, he_init)
            #x_hat_sigmoid = tf.sigmoid(x_hat_logits)
        return x_hat_logits


    def estimation_network(z_concat, K_dims, n_layer1, dropout_prob_paper):
        n_softmax=K_dims

        with tf.variable_scope("estimation"):
            init_xavier = tf.contrib.layers.xavier_initializer()
            he_init = tf.contrib.layers.variance_scaling_initializer()
            layer1 = tf.layers.dense(z_concat, n_layer1, tf.nn.tanh, he_init)
            layer1_drop = tf.layers.dropout(layer1, dropout_prob_paper, training=training)
            gamma = tf.layers.dense(layer1_drop, n_softmax, tf.nn.softmax, he_init)
            
        return gamma

    def compression_autoencoder(x, x_dim, z_dim, n_hidden1, n_hidden2, n_hidden3, dropout_prob):

        z_comp = encoder(x, z_dim, n_hidden1, n_hidden2, n_hidden3, dropout_prob)
        x_hat = decoder(z_comp, x_dim, n_hidden1, n_hidden2, n_hidden3, dropout_prob)

        ae_loss = tf.reduce_mean(tf.squared_difference(x,x_hat)) 
        recon_error = x - x_hat
        #cos_similarity and recon_norm are shape = (batch_size,)
        recon_error_magnitude = tf.norm(
                recon_error, ord='euclidean', axis=1, keep_dims=True) #tensor of shape (batch_size, 1)
        x_unit = tf.nn.l2_normalize(x, dim=1)        
        x_hat_unit = tf.nn.l2_normalize(x_hat, dim=1)
        cos_similarity=tf.reduce_sum(
                   tf.multiply(x_unit ,x_hat_unit),axis=1,keep_dims=True) #shape = (batch_size,1)

        return z_comp, x_hat, ae_loss, recon_error_magnitude, cos_similarity


    def density_parameters(gamma, z):
        #gamma expected shape-> (N,K)
        #z expected shape-> (N, z_dim)
        phi = tf.reduce_mean(gamma, axis=0)   #shape phi= (K,)

        gamma_expand = tf.expand_dims(gamma, axis=-1)  #gamma_expand -> (N,K,1)
        z_expand =  tf.expand_dims(z, axis=1)          #z_expand -> (N,1,3)
        gamma_z = tf.matmul(gamma_expand,z_expand)   #out_shape =(N,K,1)x(N,1,3)=(N,K,3) for z_dim = 3
        mu_numerator = tf.reduce_sum(gamma_z, axis=0)   #shape = (K,3)

        mu_denominator = tf.expand_dims(tf.reduce_sum(gamma, axis=0), axis=-1)   #shape = (K,1)
        mu = mu_numerator / mu_denominator  #mu shape->(K,3)
        #######################################################
        #Co-variance:
        z_expand = tf.expand_dims(z, axis=1)  #output shape=(N=2,1,3)
        mu_expand = tf.expand_dims(mu, axis=0) #output shape=(1,4,3)
        z_minus_mu = tf.subtract(z_expand, mu_expand)    #output shape=(N=2,4,3)

        z_minus_mu_expand_left = tf.expand_dims(z_minus_mu, axis=2)   #shape=(N=2,4,1,3)
        z_minus_mu_expand_right = tf.expand_dims(z_minus_mu, axis=-1)  #shape=(N=2,4,3,1)

        z_mu_outer_product = tf.matmul(z_minus_mu_expand_right, z_minus_mu_expand_left)

        gamma_double_expand =  tf.expand_dims(tf.expand_dims(gamma, axis=-1), axis=-1)

        gamma_x_outer_product = gamma_double_expand * z_mu_outer_product

        covariance_numerator = tf.reduce_sum(gamma_x_outer_product, axis=0) #shape=(K=4,3,3)

        g_expand = tf.expand_dims(tf.expand_dims(tf.reduce_sum(gamma, axis=0), axis=-1), axis=-1)

        covariance_final = covariance_numerator / g_expand  #final covariance term shape=(4,3,3) 

        return phi, mu, covariance_final

    def energy_density(z, phi, mu, covariance, z_dim, K_dims):

        z_expand = tf.expand_dims(z, axis=1)  #output shape=(N=2,1,3)
        mu_expand = tf.expand_dims(mu, axis=0) #output shape=(1,4,3)
        z_minus_mu = tf.subtract(z_expand, mu_expand)    #output shape=(N=2,4,3)

        z_minus_mu_expand_left = tf.expand_dims(z_minus_mu, axis=2) #output shape=(N=2,4,1,3)
        z_minus_mu_expand_right = tf.expand_dims(z_minus_mu, axis=-1) #output shape=(N=2,4,3,1)

        total_z_dim = z_dim + 2

        dummy_matrix = tf.eye(
            num_rows = total_z_dim, num_columns = total_z_dim, batch_shape = [K_dims], dtype=tf.float32)
        eps = 2.0e-12
        eps_matrix = dummy_matrix * eps
        covariance = tf.add(covariance, eps_matrix)


        covariance_inverse = tf.matrix_inverse(covariance)   #input and output shape = (4,3,3)

        cov_inverse_expand = tf.expand_dims(covariance_inverse, axis=0) #output shape=(1,4,3,3)
        cov_inverse_tiled = tf.tile(cov_inverse_expand, [tf.shape(z)[0],1,1,1]) #output shape=(?,4,3,3) 

        z_mu_x_cov_inverse = tf.matmul(
                                z_minus_mu_expand_left, cov_inverse_tiled)  #output shape=(2,4,1,3)

        z_mu_cov_inv_z_mu = tf.matmul(
                                z_mu_x_cov_inverse, z_minus_mu_expand_right)  #output shape=(2,4,1,1)

        exp_term = tf.exp(-0.5 * z_mu_cov_inv_z_mu)

        phi_expand = tf.expand_dims(tf.expand_dims(phi, axis=-1),axis=-1)

        phi_exp_product = phi_expand * exp_term
        phi_exp_product_squeeze = tf.squeeze(phi_exp_product)

        energy_divided_by = tf.expand_dims(
                        tf.sqrt(2.0 * 3.14159 * tf.matrix_determinant(covariance)),axis=0) + 2.0e-12

        term_to_be_summed = phi_exp_product_squeeze / tf.squeeze(energy_divided_by)

        term_inside_log = tf.reduce_sum(term_to_be_summed, axis=1) + 2.0e-12

        energy = -1.0 * tf.log(term_inside_log)
        final_energy = tf.expand_dims(energy, axis=-1)  #ouptut shape=(?=batch_size,1)

        return final_energy
    
    def test_energy(z, phi_test, mu_test, covariance_test, z_dim, K_dims):

        #data_tf = tf.convert_to_tensor(data_np, np.float32)
        #convert Numpy numerical value arrays to TF tensor objects
        phi = tf.convert_to_tensor(phi_test, np.float32)
        mu = tf.convert_to_tensor(mu_test, np.float32)
        covariance = tf.convert_to_tensor(covariance_test, np.float32)

        total_z_dim = z_dim + 2  ##########
        print("-----------", total_z_dim)

        dummy_matrix = tf.eye(
            num_rows = total_z_dim, num_columns = total_z_dim, batch_shape = [K_dims], dtype=tf.float32)
        eps = 2.0e-12
        eps_matrix = dummy_matrix * eps
        covariance = tf.add(covariance, eps_matrix)


        z_expand = tf.expand_dims(z, axis=1)  #output shape=(N=2,1,3)
        mu_expand = tf.expand_dims(mu, axis=0) #output shape=(1,4,3)
        z_minus_mu = tf.subtract(z_expand, mu_expand)    #output shape=(N=2,4,3)

        z_minus_mu_expand_left = tf.expand_dims(z_minus_mu, axis=2)   #output shape=(N=2,4,1,3)
        z_minus_mu_expand_right = tf.expand_dims(z_minus_mu, axis=-1)   #output shape=(N=2,4,3,1)


        covariance_inverse = tf.matrix_inverse(covariance)   #input and output shape = (4,3,3)

        cov_inverse_expand = tf.expand_dims(
                                     covariance_inverse, axis=0)   #output shape = (1,4,3,3)
        cov_inverse_tiled = tf.tile(
                             cov_inverse_expand, [tf.shape(z)[0],1,1,1]) #output shape=(?,4,3,3)

        z_mu_x_cov_inverse = tf.matmul(
                            z_minus_mu_expand_left, cov_inverse_tiled) #output shape=(2,4,1,3)

        z_mu_cov_inv_z_mu = tf.matmul(
                            z_mu_x_cov_inverse, z_minus_mu_expand_right) #output shape=(2,4,1,1)

        exp_term = tf.exp(-0.5 * z_mu_cov_inv_z_mu)

        phi_expand = tf.expand_dims(tf.expand_dims(phi, axis=-1),axis=-1)

        phi_exp_product = phi_expand * exp_term
        phi_exp_product_squeeze = tf.squeeze(phi_exp_product)

        
        energy_divided_by = tf.expand_dims(
                     tf.sqrt(2.0 * 3.14159 * tf.matrix_determinant(covariance)),axis=0)  + 1.0e-12

        term_to_be_summed = phi_exp_product_squeeze / tf.squeeze(energy_divided_by)

        term_inside_log = tf.reduce_sum(term_to_be_summed, axis=1) + 1.0e-12

        energy = -1.0 * tf.log(term_inside_log)
        final_energy_test = tf.expand_dims(energy, axis=-1)  #ouptut shape=(?=batch_size,1)

        return final_energy_test

    #############################################
    from sklearn.metrics import precision_recall_fscore_support as prf, accuracy_score
    def f1(thresh, energy_array, labels):
        prediction = (energy_array>thresh).astype(int)
        pred = prediction.reshape(-1)
        gt = labels.astype(int)
        accuracy = accuracy_score(gt,pred)
        precision, recall, f_score, support = prf(gt, pred, average='binary')
        return -f_score
    #######################################################################################
    x_dim  = 30   #with Time dropped there are 29 input features
    dropout_prob = tf.placeholder(tf.float32, name='dropout_prob')
    dropout_prob_paper = tf.placeholder(tf.float32, name='dropout_prob_paper')
    x = tf.placeholder(tf.float32, shape=[None, x_dim])

    z_comp, x_hat, ae_loss, recon_error_magnitude, cos_similarity = compression_autoencoder(
                            x, x_dim, z_dim, n_hidden1, n_hidden2, n_hidden3, dropout_prob)

    z_concat =  tf.concat([z_comp, recon_error_magnitude, cos_similarity ], axis=1)

    gamma = estimation_network(z_concat, K_dims, n_layer1, dropout_prob_paper)

    phi, mu, covariance = density_parameters(gamma, z_concat)

    energy = energy_density(z_concat, phi, mu, covariance, z_dim, K_dims)

    energy_loss = tf.reduce_mean(energy)

    indices = [[k_index, j_index, j_index] for k_index in range(K_dims) 
               for j_index in range(z_dim + 2)]

    #indices = [[0,0,0], [0,1,1], [0,2,2], [1,0,0], [1,1,1], 
                       #[1,2,2], [2,0,0], [2,1,1], [2,2,2], [3,0,0], [3,1,1], [3,2,2]]

    cov_loss = tf.gather_nd(covariance, indices)
    cov_loss_reciprocal = 1 / cov_loss
    P_Sigma_loss = tf.reduce_sum(cov_loss_reciprocal)

    total_loss = ae_loss + lamda1*energy_loss + lamda2*P_Sigma_loss 

    t_vars = tf.trainable_variables()
    all_training_vars = [var for var in t_vars if "encoder" or "decoder" or "estimation" in var.name]

    training_op = tf.train.AdamOptimizer(lr).minimize(total_loss, var_list=all_training_vars)

    number_of_batches = int(total_training_instances / batch_size)

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer(), 
                             feed_dict={dropout_prob : hp_drop, dropout_prob_paper : hp_drop})

        for epoch in range(n_epochs):
            
            '''# Random shuffling
            np.random.shuffle(train_total_data_normal)
            train_data_ = train_total_data_normal[:, 0:x_dim]   
            train_label_ = train_total_data_normal[:, x_dim]
            #test #################################
            test_data = test_total_data[:,0:x_dim]
            test_labels = test_total_data[:,x_dim]'''
            
            train_data_ = train_total_data_normal
            test_data = test_total_data_normal
            test_labels = y_test_normal

            #######################################
            # Loop over all batches
            for i in range(number_of_batches):
                # Compute the offset of the current minibatch in the data.
                offset = (i * batch_size) % (total_training_instances)
                batch_x_input = train_data_[offset:(offset + batch_size)]
                
          
                _, ae_loss_val, energy_loss_val, p_sigma_loss_val, total_loss_val = sess.run(
                    (training_op, ae_loss, energy_loss, P_Sigma_loss, total_loss),
                    feed_dict={x: batch_x_input, dropout_prob : hp_drop, 
                               dropout_prob_paper : hp_drop, training: True})
        
            #print("epoch = ",epoch, "batch_number = ", i, 
                  #"ae_loss=", ae_loss_val, "energy_loss=", energy_loss_val,
                  #"p_sigma_loss_val=", p_sigma_loss_val, "total_loss=", total_loss_val)
        
            #evaluate energy of total training dataset each training step
            energy_loss_val_entire = sess.run((energy_loss), feed_dict={x: train_data_ , 
                                dropout_prob : 0.0, dropout_prob_paper : 0.0, training: False} )
            #print("energy loss entire batch=", energy_loss_val_entire)
            '''
            Training done;  Start testing
            TO DO
            1) Calculate fixed phi, mu, and covariance derived on whole training data set
            2) Define "Test_Energy" function using fixed values above in (1) and taking input points 
            to give Energy this can calculate energy of 1 point or a batch.
            3) Use this to calaculate energy of each data point (normal and anomaly) in test dataset. 
            If normal/anomaly ratio is 4:1 then define an anomaly as the 80th percentile 
            energy threshold (find this threshold)
            4) Using this threshold and entire test dataset, 
            calcultate predcision-recall curves, F1 score, etc...
            #########################################
            '''
        #1)
        phi_test_val, mu_test_val, cov_test_val = sess.run((phi, mu, covariance), 
                feed_dict={x: train_data_ , dropout_prob : 0.0, 
                           dropout_prob_paper : 0.0, training: False} )
        #2) 
        energy_test = test_energy(z_concat, phi_test_val , mu_test_val, cov_test_val, z_dim, K_dims)
        #3)  
        energy_test_val = sess.run((energy_test), 
                                   feed_dict={x: test_data , dropout_prob : 0.0, 
                                              dropout_prob_paper : 0.0, training: False} )
     

    result_val = minimize_scalar(f1, args=(energy_test_val,test_labels))
    return result_val.fun  #note:  negative of F1-score is returned

shape train_total_data_normal (113842, 30)
113842


In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

neg_f1_score = dagmm(n_epochs=50, z_dim=1,K_dims=1,lamda1=0.1,lamda2=0.01,lr=0.0001,batch_size=1024)
print("neg F1 score result=", neg_f1_score)

#use scikit optimize (skopt) function gp_minimize to perfrom hyperparamter optimization

#import pickle
import numpy as np
import skopt
from skopt import gp_minimize

def main(params):
    
    n_epochs, lamda1, lamda2, lr, batch_size, n_hidden1, n_hidden2, n_hidden3, n_layer1 = params
    print("params:n_epochs, l1, l2, lr, batch_size, n_h1, n_h2, n_h3, n_l1 :", params)
    
    neg_f1_score = dagmm(n_epochs=n_epochs, z_dim=1,K_dims=3,lamda1=lamda1,
                         lamda2=lamda2,lr=lr,batch_size=batch_size,n_hidden1=n_hidden1, 
                         n_hidden2=n_hidden2, n_hidden3=n_hidden3, n_layer1=n_layer1)
    return neg_f1_score

if __name__ == "__main__":
    params = [
        (7, 100),
        np.logspace(-2, 0, 10),
        np.logspace(-3, -1, 10),
        np.logspace(-4, -1, 10),
        (256, 2000),
        (10, 100),
        (10, 100),
        (3, 10),
        (5,30)
    ]
    res = gp_minimize(func=main, dimensions=params, n_calls=300, verbose=True)

    print(res.x, res.fun)
    #pickle.dump(res, open('res.pkl', 'wb'))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


----------- 3
neg F1 score result= -0.13414634146341464
Iteration No: 1 started. Evaluating function at random point.
params:n_epochs, l1, l2, lr, batch_size, n_h1, n_h2, n_h3, n_l1 : [67, 0.027825594022071243, 0.0016681005372000592, 0.00046415888336127773, 1572, 85, 12, 5, 25]
----------- 3
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 129.5691
Function value obtained: -0.1026
Current minimum: -0.1026
Iteration No: 2 started. Evaluating function at random point.
params:n_epochs, l1, l2, lr, batch_size, n_h1, n_h2, n_h3, n_l1 : [16, 0.5994842503189409, 0.0027825594022071257, 0.004641588833612777, 536, 31, 85, 4, 25]
----------- 3
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 42.8874
Function value obtained: -0.1263
Current minimum: -0.1263
Iteration No: 3 started. Evaluating function at random point.
params:n_epochs, l1, l2, lr, batch_size, n_h1, n_h2, n_h3, n_l1 : [43, 0.027825594022071243, 0.004641588833612777, 0.021544346900318822, 1727, 5